In [1]:
%cd ../../

/home/PJWSTK/s14028/engineer/Practical


In [2]:
import os

import pandas as pd
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from testing import test_model
from testing import best_cop_diff
from testing import mse
from testing import mae

from CNN.CNN_v8 import CNN_v8

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn = CNN_v8((480, 640, 4), split_into_parts=20)

images = cnn._prepare_images(image_test)
anwsers = cnn._prepare_anwsers(person_coo_test)

In [5]:
cop = np.linspace(0, 1, 11)[1:-1]

In [6]:
prefix = "CNN/CNN_v8/weights/"

def get_weights_paths(prefix):
    for directory, subdirectories, files in os.walk(prefix):
        yield from (os.path.join(directory, file) for file in files)

model_names = get_weights_paths(prefix)
model_names = list(model_names)

In [7]:
models = [CNN_v8((480, 640, 4), split_into_parts=20) for i in range(len(model_names))]

In [8]:
results = {}

for model_name, model in zip(model_names, models):
  model.def_model()
  model.model.fit(images[:1], anwsers[:1])
  
  model.model = keras.utils.multi_gpu_model(model.model, gpus=2, cpu_merge=False)
  model.model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  
  model.model.load_weights(model_name)
  
  results[model_name] = test_model(model=model, images=image_test, counts=count_test, cut_of_points=cop)

Epoch 1/1
1/1 [==============================] - 2s 2s/step - loss: 0.6967 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 658ms/step - loss: 0.6960 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 681ms/step - loss: 0.6968 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 689ms/step - loss: 0.6958 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 707ms/step - loss: 0.6965 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 694ms/step - loss: 0.6970 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 677ms/step - loss: 0.6964 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 681ms/step - loss: 0.6965 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 671ms/step - loss: 0.6957 - accuracy: 1.0000
Epoch 1/1
1/1 [==============================] - 1s 665ms/step - loss: 0.6956 - accuracy: 1.0000
Epoch 1/1
1/1 [==================

In [9]:
def as_rows(results):
    for name, cut_off_point_to_accuracy_with_difference in results.items():
        yield from ((f"{name}_{cut_off_point}", accuracy, counts - count_test) for cut_off_point, (accuracy, counts) in cut_off_point_to_accuracy_with_difference.items())

rows = as_rows(results)
rows = list(rows)

In [10]:
def calculate_mae_and_mse(rows):
    for *attributes, difference in rows:
        difference_mae = mae(difference)
        difference_mse = mse(difference)
        
        yield (*attributes, difference_mae, difference_mse)

mae_and_mse = calculate_mae_and_mse(rows)
mae_and_mse = list(mae_and_mse)

In [11]:
frame = pd.DataFrame(mae_and_mse, columns=["model", "accuracy", "mae", "mse"])
frame

,model,accuracy,mae,mse
0,CNN/CNN_v8/weights/cnn_v8_1e_6_5_0.1,0.000000,345.653333,119568.833333
1,CNN/CNN_v8/weights/cnn_v8_1e_6_5_0.2,0.000000,234.943333,55717.723333
2,CNN/CNN_v8/weights/cnn_v8_1e_6_5_0.30000000000...,0.000000,152.920000,23777.333333
3,CNN/CNN_v8/weights/cnn_v8_1e_6_5_0.4,0.000000,103.096667,10927.416667
4,CNN/CNN_v8/weights/cnn_v8_1e_6_5_0.5,0.033333,58.276667,3581.563333
...,...,...,...,...
445,CNN/CNN_v8/weights/cnn_v8_1e_6_54_0.5,81.182912,5.933333,47.986667
446,CNN/CNN_v8/weights/cnn_v8_1e_6_54_0.6000000000...,69.849281,9.473333,105.720000
447,CNN/CNN_v8/weights/cnn_v8_1e_6_54_0.7000000000...,58.685242,12.916667,186.383333
448,CNN/CNN_v8/weights/cnn_v8_1e_6_54_0.8,46.992572,16.513333,296.626667


In [12]:
frame.loc[frame.mae == frame.mae.min()]

,model,accuracy,mae,mse
219,CNN/CNN_v8/weights/cnn_v8_1e_6_29_0.4,90.280334,2.996667,13.743333


In [13]:
frame.loc[frame.mse == frame.mse.min()]

,model,accuracy,mae,mse
219,CNN/CNN_v8/weights/cnn_v8_1e_6_29_0.4,90.280334,2.996667,13.743333


In [14]:
frame.loc[frame.accuracy == frame.accuracy.max()]

,model,accuracy,mae,mse
219,CNN/CNN_v8/weights/cnn_v8_1e_6_29_0.4,90.280334,2.996667,13.743333
